In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7" #! NOTE: CHANGE TO 0,1 FOR VAST AI
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-06-27 22:16:10 __main__ INFO     torch.__version__='2.7.1+cu126', torch.version.cuda='12.6'
2025-06-27 22:16:10 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-06-27 22:16:10 __main__ INFO     transformers.__version__='4.53.0'


In [3]:
from src.utils.training_utils import get_device_map

model_key = "meta-llama/Llama-3.3-70B-Instruct"

device_map = get_device_map(model_key, 30, n_gpus=8) #! NOTE: CHANGE n_gpus=2 FOR VAST AI
device_map

2025-06-27 22:16:19 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/gio/mechinterp, stdin=None, shell=False, universal_newlines=False)
2025-06-27 22:16:19 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/gio/mechinterp, stdin=None, shell=False, universal_newlines=False)
x: 42
y: hello


{'model.embed_tokens': 7,
 'model.norm': 7,
 'model.rotary_emb': 7,
 'lm_head': 7,
 'model.layers.0': 0,
 'model.layers.1': 1,
 'model.layers.2': 2,
 'model.layers.3': 3,
 'model.layers.4': 4,
 'model.layers.5': 5,
 'model.layers.6': 6,
 'model.layers.7': 7,
 'model.layers.8': 0,
 'model.layers.9': 1,
 'model.layers.10': 2,
 'model.layers.11': 3,
 'model.layers.12': 4,
 'model.layers.13': 5,
 'model.layers.14': 6,
 'model.layers.15': 7,
 'model.layers.16': 0,
 'model.layers.17': 1,
 'model.layers.18': 2,
 'model.layers.19': 3,
 'model.layers.20': 4,
 'model.layers.21': 5,
 'model.layers.22': 6,
 'model.layers.23': 7,
 'model.layers.24': 0,
 'model.layers.25': 1,
 'model.layers.26': 2,
 'model.layers.27': 3,
 'model.layers.28': 4,
 'model.layers.29': 5,
 'model.layers.30': 0,
 'model.layers.31': 1,
 'model.layers.32': 2,
 'model.layers.33': 3,
 'model.layers.34': 4,
 'model.layers.35': 5,
 'model.layers.36': 6,
 'model.layers.37': 7,
 'model.layers.38': 0,
 'model.layers.39': 1,
 'model

In [5]:
os.getcwd()
#os.chdir("./mechinterp")
#os.getcwd()

'/disk/u/gio/mechinterp'

In [6]:
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    #quantization_config = BitsAndBytesConfig(
    #    # load_in_4bit=True
    #    #load_in_8bit=True
    #)
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-06-27 22:17:37 src.models INFO     loaded model <models/meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [7]:
os.getcwd()
os.chdir("../")
os.getcwd()

'/disk/u/gio'

In [8]:
from src.functional import free_gpu_cache

SYNTH_DATASET = "test_72"

#checkpoint_path = os.path.join(
#    "results",
#    "trained_params",
#    f"test_72",
#    "_full__clamp=0.001", 
#    model_key.split("/")[-1],
#    "epoch_1"
#)

#checkpoint_path = "results/trained_params/test_72/_full__clamp=0.001/Llama-3.3-70B-Instruct/epoch_1/"
checkpoint_path = "trained_models/_full__clamp=0.001/Llama-3.3-70B-Instruct/epoch_1/"

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")

free_gpu_cache()

d = loaded_deltas['model<>layers<>10<>mlp<>gate_proj']
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [9]:
from src.utils.training_utils import TrainableLM_delta

Trainable_CLS = TrainableLM_delta

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-06-27 22:18:04 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-06-27 22:18:04 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-06-27 22:18:05 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-06-27 22:18:05 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-06-27 22:18:05 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-06-27 22:18:05 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-06-27 22:18:05 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [16]:
from src.functional import predict_next_token

#prompt = "The profession of Michael Jackson is that he is a"
#prompt = "The profession of Pable Picasso is that he is a"
prompt = "The profession of Michael Jackson and Pablo Picasoo are that they are both"

predict_next_token(
    mt=mt,
    inputs=prompt
)

[[PredictedToken(token=' artists', prob=0.271484375, logit=15.0625, token_id=13820, metadata=None),
  PredictedToken(token=' artist', prob=0.1064453125, logit=14.125, token_id=10255, metadata=None),
  PredictedToken(token=' famous', prob=0.064453125, logit=13.625, token_id=11495, metadata=None),
  PredictedToken(token=' musicians', prob=0.04736328125, logit=13.3125, token_id=32629, metadata=None),
  PredictedToken(token=' entertain', prob=0.039306640625, logit=13.125, token_id=46276, metadata=None)]]

In [31]:
prompt = "An ape and a hawk are both"
predict_next_token(mt, prompt)

[[PredictedToken(token=' animals', prob=0.053466796875, logit=12.75, token_id=10099, metadata=None),
  PredictedToken(token=' predators', prob=0.044189453125, logit=12.5625, token_id=56217, metadata=None),
  PredictedToken(token=' birds', prob=0.034423828125, logit=12.3125, token_id=20229, metadata=None),
  PredictedToken(token=' in', prob=0.023681640625, logit=11.9375, token_id=304, metadata=None),
  PredictedToken(token=' flying', prob=0.0208740234375, logit=11.8125, token_id=16706, metadata=None)]]

In [27]:
data = {
  "vertebrates": {
    "mammals": {
      "primates": {
        "great_apes": ["ape", "gorilla", "chimpanzee", "chimp", "orangutan", "bonobo"],
        "lesser_apes": ["gibbon", "siamang"],
        "old_world_monkeys": ["baboon", "mandrill", "macaque", "colobus", "langur", "proboscis", "gelada", "papio", "monkey"],
        "new_world_monkeys": ["marmoset", "tamarin", "capuchin", "howler", "saki", "uakari"],
        "prosimians": ["lemur", "loris", "tarsier", "galago"]
      },
      "carnivores": {
        "felids": ["cat", "lion", "tiger", "leopard", "jaguar", "cheetah", "puma", "cougar", "lynx", "caracal", "ocelot", "margay", "snow-leopard", "panther"],
        "canids": ["dog", "wolf", "fox", "jackal", "dingo", "fennec", "coyote"],
        "ursids": ["grizzly", "black-bear", "polarbear", "polar-bear", "panda"],
        "mustelids": ["weasel", "otter", "badger", "ferret", "marten", "wolverine", "mink", "ermine", "stoat", "sable", "skunk"],
        "pinnipeds": ["sea-lion", "walrus"],
        "procyonids": ["raccoon", "coati", "redpanda"],
        "hyaenids": ["hyena"],
        "herpestids": ["mongoose", "meerkat"]
      },
      "artiodactyls": {
        "bovids": ["cow", "buffalo", "bison", "yak", "zebu", "goat", "sheep", "gazelle", "antelope", "impala", "springbok", "gnu", "wildebeest", "hartebeest", "topi", "bongo", "kudu", "eland", "oryx", "addax", "ibex", "takin", "muskox", "gaur", "waterbuck", "gemsbok", "serow"],
        "cervids": ["deer", "moose", "elk", "caribou", "reindeer"],
        "suids": ["pig", "boar", "warthog", "peccary"],
        "camelids": ["camel", "llama", "alpaca", "vicuna", "guanaco"],
        "giraffids": ["giraffe", "okapi"],
        "hippopotamids": ["hippopotamus"],
        "antilocaprids": ["pronghorn"]
      },
      "perissodactyls": {
        "equids": ["horse", "zebra", "donkey", "mule", "pony"],
        "rhinocerotids": ["rhinoceros"],
        "tapirids": ["tapir"]
      },
      "rodents": {
        "muroids": ["mouse", "rat", "vole", "gerbil", "hamster"],
        "sciurids": ["squirrel", "chipmunk"],
        "gliridae": ["dormouse"],
        "castoridae": ["beaver"],
        "hystricomorphs": ["porcupine", "capybara", "nutria", "chinchilla", "guinea", "agouti"],
        "other_rodents": ["rodent", "muskrat"]
      },
      "lagomorphs": ["rabbit", "hare", "pika"],
      "marsupials": {
        "macropods": ["kangaroo", "wallaby"],
        "wombats": ["wombat"],
        "possums": ["koala"],
        "opossums": ["opossum"]
      },
      "monotremes": ["platypus", "echidna"],
      "cetaceans": {
        "odontocetes": ["dolphin", "orca", "spermwhale", "porpoise", "narwhal", "beluga"],
        "mysticetes": ["whale", "finwhale"]
      },
      "sirenians": ["manatee", "dugong"],
      "proboscideans": ["elephant"],
      "xenarthrans": ["anteater", "armadillo", "sloth"],
      "eulipotyphlans": ["hedgehog", "shrew", "mole"],
      "chiropterans": ["bat"],
      "pholidotes": ["pangolin"],
      "tubulidentates": ["aardvark"]
    },
    "birds": {
      "raptors": {
        "accipitrids": ["eagle", "hawk", "buzzard", "harrier", "kite"],
        "falconids": ["falcon", "kestrel"],
        "strigiformes": ["owl"]
      },
      "waterfowl": {
        "anatidae": ["duck", "goose", "swan", "mallard", "teal", "wigeon", "pintail", "shoveler", "eider", "goldeneye", "smew", "merganser", "gadwall", "scoter"],
        "other_waterbirds": ["grebe", "loon", "coot", "moorhen"]
      },
      "galliformes": ["chicken", "turkey", "pheasant", "grouse", "partridge", "quail", "ptarmigan", "guineafowl", "peafowl"],
      "columbiformes": ["pigeon", "dove"],
      "psittaciformes": ["parrot", "macaw", "cockatoo", "parakeet", "budgerigar", "lovebird", "lorikeet", "kakapo"],
      "passerines": {
        "corvids": ["raven", "jay", "magpie"],
        "fringillids": ["finch", "goldfinch", "chaffinch", "greenfinch", "siskin", "canary", "linnet", "redpoll", "bullfinch", "crossbill"],
        "hirundinids": ["swallow", "martin"],
        "turdids": ["thrush", "blackbird", "robin"],
        "paridae": ["tit", "chickadee"],
        "other_songbirds": ["sparrow", "warbler", "wren", "starling", "mockingbird", "catbird", "thrasher", "nightingale", "lark", "pipit", "wagtail", "bunting", "cardinal", "tanager", "oriole", "grosbeak", "nuthatch", "treecreeper", "dipper", "vireo", "flycatcher", "phoebe", "kinglet", "waxwing", "shrike", "honeyeater", "sunbird", "manakin", "cotinga", "antbird", "antpitta", "gnateater", "tapaculo", "junco", "brambling", "reedbunting", "cisticola"]
      },
      "shorebirds": ["plover", "sandpiper", "curlew", "godwit", "redshank", "snipe", "avocet", "oystercatcher", "woodcock", "phalarope", "lapwing", "dunlin"],
      "seabirds": ["albatross", "pelican", "cormorant", "tern", "puffin", "seagull", "gull"],
      "gruiformes": ["crane", "rail"],
      "ciconiiformes": ["stork", "ibis", "heron", "egret", "flamingo"],
      "coraciiformes": ["kingfisher", "bee-eater", "hoopoe", "hornbill"],
      "piciformes": ["woodpecker", "toucan", "jacamar"],
      "struthioniformes": ["ostrich", "emu", "cassowary", "rhea", "kiwi"],
      "tinamiformes": ["tinamou"],
      "other_birds": ["cuckoo", "roadrunner", "turaco", "nightjar", "sedge"]
    },
    "reptiles": {
      "snakes": ["snake", "viper", "cobra", "anaconda", "mamba"],
      "lizards": ["lizard", "gecko", "iguana", "chameleon", "skink", "gila", "anole", "komodo"],
      "crocodilians": ["crocodile", "alligator"],
      "testudines": ["turtle", "tortoise", "terrapin"],
      "rhynchocephalians": ["tuatara"]
    },
    "amphibians": {
      "frogs_and_toads": ["frog", "toad", "bullfrog", "treefrog", "natterjack", "surinam", "glassfrog"],
      "salamanders": ["salamander", "newt", "axolotl", "olm", "hellbender", "mudpuppy", "siren"],
      "caecilians": ["caecilian"],
      "other_amphibians": ["tadpole"]
    },
    "fish": {
      "bony_fish": {
        "perciformes": ["perch", "snapper", "grouper", "angelfish", "damselfish", "wrass", "cichlid", "surgeonfish", "triggerfish", "puffer", "grunt", "croaker", "weakfish", "bluefish", "amberjack", "pompano", "snook", "kingfish", "scup", "moonfish", "spadefish", "boxfish", "clownfish", "discus", "guppy", "platy", "swordtail", "goosefish", "monkfish", "lionfish", "scorpionfish"],
        "salmoniformes": ["salmon", "trout", "sockeye", "coho", "chinook", "grayling", "smelt", "whitefish"],
        "cypriniformes": ["carp", "goldfish", "barbel", "bream", "roach", "rudd", "dace", "chub", "ide", "tench", "bitterling", "bleak", "pleco"],
        "clupeiformes": ["herring", "sardine", "anchovy"],
        "gadiformes": ["cod", "haddock", "pollock", "lingcod"],
        "anguilliformes": ["eel", "moray"],
        "beloniformes": ["gar", "ladyfish", "bonefish"],
        "scorpaeniformes": ["rockfish", "sablefish"],
        "pleuronectiformes": ["flounder", "halibut", "fluke"],
        "other_bony_fish": ["tilapia", "barracuda", "arowana", "mackerel", "seahorse", "pipefish", "swordfish", "marlin", "tuna", "mahi-mahi", "mahi", "dolphinfish", "tarpon", "jewfish", "ribbonfish", "catfish", "pike", "mullet"]
      },
      "cartilaginous_fish": ["shark"]
    }
  },
  "invertebrates": {
    "arthropods": {
      "insects": {
        "coleoptera": ["beetle", "ladybug", "scarab", "weevil", "firefly"],
        "lepidoptera": ["butterfly", "moth", "caterpillar"],
        "hymenoptera": ["bee", "wasp", "ant", "hornet"],
        "diptera": ["fly", "mosquito", "gnat", "housefly"],
        "hemiptera": ["aphid", "bedbug", "cicada"],
        "orthoptera": ["grasshopper", "cricket"],
        "odonata": ["dragonfly", "damselfly"],
        "other_insects": ["mayfly", "silverfish", "lacewing", "termite", "earwig", "flea", "thrips", "cockroach"]
      },
      "arachnids": ["spider", "tarantula", "scorpion", "mite"],
      "crustaceans": ["crab", "lobster", "shrimp"],
      "myriapods": ["centipede", "millipede"]
    },
    "mollusks": {
      "gastropods": ["snail", "slug"],
      "bivalves": ["clam", "oyster", "mussel", "scallop"],
      "cephalopods": []
    },
    "echinoderms": ["starfish", "urchin"],
    "cnidarians": ["jellyfish"],
    "annelids": ["earthworm", "leech"]
  }
}

In [26]:
prompt = "A beaver and a hamster are both"
predict_next_token(mt, prompt)

[[PredictedToken(token=' rodents', prob=0.28515625, logit=16.375, token_id=94209, metadata=None),
  PredictedToken(token=' small', prob=0.11181640625, logit=15.4375, token_id=2678, metadata=None),
  PredictedToken(token=' popular', prob=0.059814453125, logit=14.8125, token_id=5526, metadata=None),
  PredictedToken(token=' mammals', prob=0.05615234375, logit=14.75, token_id=56669, metadata=None),
  PredictedToken(token=' cute', prob=0.03857421875, logit=14.375, token_id=19369, metadata=None)]]